In [1]:
from platypus import NSGAII, Problem, Real
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors
import pandas as pd
import random

In [2]:
# Load the autoreload extension
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [3]:
from Mult_Obj_Helathy import Main_Model as Mdl_H 
from Mult_Obj_Tumor import Main_Model as Mdl_T

# Mixture of CAR NK and WT
(1) check for 50 and 50 

In [4]:
Shell_H = Mdl_H()
Shell_T = Mdl_T()

In [5]:
Tumor_No = 10000
for i in [10.0,5.0,2.5,1.25,0.625,0.3125]:
    x0 = i
    NK_No = x0*Tumor_No
    x = [0.6,0.6]
    param_100 = np.array([0.01, 1.3611247151766386, 2.646954531914279, 1.119928427439465, 0.8030447162325958]) 
    frac = param_100[-1]
    MFI_limt = [[(0, 100), (0, 300), (0.0, 50.0)], [(0, 30), (0, 70), (0, 20.0)]]
    Shell_T.Tumor(x[0], frac, x[1], NK_No = NK_No, Tc_No = Tumor_No, param=param_100[:-1], NK_cell='MIX', Tumor_cell='Kasumi1',limts=MFI_limt)
    MFI_limt = [[(0, 100), (0, 300), (0.0, 50.0)], [(0, 40), (0, 45), (0, 200.0)]]
    Shell_H.Healthy(x[0], frac, x[1], NK_No = NK_No, Tc_No = Tumor_No, param=param_100[:-1], NK_cell='MIX', Tumor_cell='Monocyte',limts=MFI_limt)
    print('done')

T 55.17729285596309
H 68.54138044393432
done
T 43.937907154653665
H 57.53149268480843
done
T 32.558357336859814
H 45.565973110299616
done
T 21.49881666412479
H 32.97488867943984
done
T 12.720098420954306
H 21.48442282467512
done
T 6.9715559485236
H 12.609310147255348
done


In [5]:
# Define your multi-objective problem here
class MyProblem(Problem):
    def __init__(self,ET_ratio):
        super(MyProblem, self).__init__(1, 2) # #variables, #objectives)
        self.types[:] = [Real(0.0, 1.0)]
        self.Shell_H = Mdl_H()
        self.Shell_T = Mdl_T()
        self.ET_ratio = ET_ratio
        self.directions[:] = [Problem.MINIMIZE, Problem.MAXIMIZE]

    def evaluate(self, solution):
        x = np.array(solution.variables)
        print('x0',x)
        Tumor_No = 10000
        NK_No = self.ET_ratio*Tumor_No
        param_100 = np.array([0.01, 1.3611247151766386, 2.646954531914279, 1.119928427439465, 0.8030447162325958])
        frac = param_100[-1]
        alpha4 = 0.6
        
        MFI_limt = [[(0, 100), (0, 300), (0.0, 50.0)], [(0, 40), (0, 45), (0, 200.0)]]
        f1 = self.Shell_H.Healthy(x[0], frac, alpha4, NK_No = NK_No, Tc_No = Tumor_No, param=param_100[:-1], NK_cell='MIX', Tumor_cell='Monocyte',limts=MFI_limt) # Objective 1
        MFI_limt = [[(0, 100), (0, 300), (0.0, 50.0)], [(0, 30), (0, 70), (0, 20.0)]]
        f2 = self.Shell_T.Tumor(x[0], frac, alpha4, NK_No = NK_No, Tc_No = Tumor_No, param=param_100[:-1], NK_cell='MIX', Tumor_cell='Kasumi1',limts=MFI_limt) # Objective 2
        solution.objectives[:] = [f1, f2]

In [ ]:
seed_value = 42
np.random.seed(seed_value)
random.seed(seed_value)
for ET_ratio in [1,5,10]:
        problem = MyProblem(ET_ratio)
        algorithm = NSGAII(problem)
        algorithm.run(2)
        Healthy_Lysis = [s.objectives[0] for s in algorithm.result]
        Tumor_Lysis = [s.objectives[1] for s in algorithm.result]
        alpha1 =  [s.variables[0] for s in algorithm.result]
        filename = f'a1_Pfront_Kasumi_Healthy_CAR_NK_ET_{ET_ratio}.xlsx'
        df = pd.DataFrame(columns=['alpha1','Tumor_lysis', 'Healthy_Lysis'])
        df['alpha1'] = alpha1
        df['Tumor_lysis'] = Tumor_Lysis
        df['Healthy_Lysis'] = Healthy_Lysis
        df.to_excel(filename)
        print('Done for', ET_ratio)